In [ ]:
import os, glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from haversine import haversine
import datetime
import math
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig
from shapely.geometry import LineString,shape, mapping

network = Network("small-Sanf/edges.shp","fid", "u", "v")
graph = NetworkGraph(network)

ubodt = UBODT.read_ubodt_csv("small-Sanf/ubodt.txt")
fmm_model = FastMapMatch(network,graph,ubodt)

k = 8
radius = 0.003
gps_error = 0.0005
fmm_config = FastMapMatchConfig(k,radius,gps_error)

path = os.getcwd() +'data/cabspottingdata/Multiple_Trajectories/Single_Trajectory/'
file_list = os.listdir(path)
f_name=[]
for filename in file_list:
    if os.path.splitext(filename)[1] == '.csv':
        f_name.append(os.path.splitext(filename)[0])
        

for t in tqdm(f_name):
    df = pd.read_csv(path + t + '.csv', index_col = 0)
    
    for wkt, time, cnt in zip(df['LineString'], df['StartTime'], df['Num_p']):
        result = fmm_model.match_wkt(wkt, fmm_config)
        
        candidates = []
        for c in result.candidates:
            candidates.append((c.spdist))
            
        df_c = pd.DataFrame(candidates, columns=["spdist"])
        travel_len = df_c['spdist'].sum()*100000     
        
        temp_geo = ''
        temp_point = ''
        temp_cpath = ''
        
        for z in list(result.cpath):
            temp_cpath += str(z) + ', '
        temp_cpath = temp_cpath[:-2]
        
        startTime = time[11:]
        startTime = startTime.split(':')            
        
        temp_geo = result.mgeom.export_wkt()
        temp_point = result.pgeom.export_wkt()
        
        df2 = df2.append({'Cpath': temp_cpath,
                'StartTime' : ti,
                'geometry' : temp_geo,
                'point' : temp_point,
                'travel_length' : travel_len,
                'travel_time' : cnt,
                'time' : startTime[0]}, ignore_index=True)
df2.to_csv(path + 'matched/alltrajectory.csv', index=False)